In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
DIRS=[]
Dir="/DIR/CSVs_CrowdTangle/"
dirs=os.listdir(r"/DIR/CSVs_CrowdTangle")
for i in dirs:
  DIRS.append(Dir+i)
#print(DIRS)

In [ ]:
#Loading the sentiment analysis model
!pip install --upgrade pip
!pip install sentencepiece
!pip install transformers
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification
from matplotlib import pyplot as plt;from matplotlib.font_manager import FontProperties
import numpy as np;from scipy.special import softmax;import pandas as pd

from google.colab import drive
drive.mount('/content/drive') #standard drive mounting

CUDA = True # set to true if using GPU (Runtime -> Change runtime Type -> GPU)
BATCH_SIZE = 32;n_labels = 3;labels_ids = {'negative': 0, 'neutral': 1, 'positive':2};model_max_length = 650

MODEL = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL, use_fast=True);config = AutoConfig.from_pretrained(MODEL);model = AutoModelForSequenceClassification.from_pretrained(MODEL)
tokenizer.model_max_length=model_max_length

if CUDA:
  model = model.to('cuda') #This sends the parameters to the GPU assigned in CUDA = True
_ = model.eval()


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 93.2 MB/s eta 0:00:00
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def forward_sentiment(text, cuda=True):
  encoded_input = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
  if cuda:
    encoded_input.to('cuda')
    output = model(**encoded_input);
    scores = output[0].detach().cpu().numpy() #cuda parametrisation
  else:
    output = model(**encoded_input);scores = output[0].detach().numpy()

  scores = softmax(scores, axis=-1);return scores

def sentiment_pred_score (data_list):
  data_list_2=[]
  sentiment_pred = []
  for i in data_list:
    try:
      i=str(i )
      if len(i)<1500:
        data_list_2.append(i)
      else:
        data_list_2.append("NAN")
    except:
      pass
  #print("Passing Data Onto The Model")
  #print(len(data_list),"ORIGINAL DATA",len(data_list_2),"AMENDED DATALIST")
  dl = DataLoader(data_list_2, batch_size=BATCH_SIZE)
  for idx, batch in enumerate(dl):

    scores = forward_sentiment(batch, cuda=CUDA);preds = np.argmax(scores, axis=-1);sentiment_pred.extend(preds)
  #print("Predictions succesfully done")
  #print(len(sentiment_pred))
  return sentiment_pred

In [ ]:
import pandas as pd
import numpy as np
TotalInteractionCountList=[]
TotalLikeList=[]
TotalCommentList=[]
TotalScoreList=[]
TotalShareList=[]
SentimentList=[]
ListOfVars=[TotalInteractionCountList,TotalLikeList,TotalCommentList,TotalScoreList,TotalShareList]
for i in DIRS:
  df=pd.read_csv(i)
  messages=df['message']
  score=df['score']
  TOTALINTERACTION=df['totalInteraction']
  LIKE=df['statistics.actual.likeCount']
  COMMENT=df['statistics.actual.commentCount']
  SHARE=df['statistics.actual.shareCount']
  TotalInteractionCountList.extend(TOTALINTERACTION)
  TotalLikeList.extend(LIKE)
  TotalScoreList.extend(score)
  TotalCommentList.extend(COMMENT)
  TotalShareList.extend(SHARE)

  sentiment=sentiment_pred_score(messages)
  SentimentList.extend(sentiment)

In [ ]:
for i in ListOfVars:
  print(len(i))
print(len(SentimentList))

35891
35891
35891
35891
35891
35891


In [ ]:
import re
for i in ListOfVars:
  templist=[]
  for j in i:
    j=str(j)
    j=re.sub(r"\,",r"",j)
    j=float(j)
    templist.append(j)
  print(np.corrcoef(SentimentList,templist))

[[ 1.        -0.1004949]
 [-0.1004949  1.       ]]
[[ 1.         -0.02685734]
 [-0.02685734  1.        ]]
[[ 1.         -0.12395784]
 [-0.12395784  1.        ]]
[[ 1.         -0.00842343]
 [-0.00842343  1.        ]]
[[ 1.         -0.13482431]
 [-0.13482431  1.        ]]
